# SPARK API MINI EXERCISES
***

In [3]:
import pandas as pd
import numpy as np
import pyspark
import pydataset
from pyspark.sql.functions import sum, mean, concat, lit, regexp_extract, regexp_replace, when
from vega_datasets import data
from pyspark.sql.functions import month, year, quarter
from pyspark.sql.functions import *

np.random.seed(13)

In [4]:
pandas_dataframe = pd.DataFrame(
    {
        "n": np.random.randn(20),
        "group": np.random.choice(list("xyz"), 20),
        "abool": np.random.choice([True, False], 20),
    }
)

In [5]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/18 11:55:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/18 11:55:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


***
1. Spark Dataframe Basics

    1. Use the starter code above to create a pandas dataframe.
    1. Convert the pandas dataframe to a spark dataframe. From this point
       forward, do all of your work with the spark dataframe, not the pandas
       dataframe.
    1. Show the first 3 rows of the dataframe.
    1. Show the first 7 rows of the dataframe.
    1. What is the difference between `.show` and `.head`?
    1. View a summary of the data using `.describe`.
    1. Use `.select` to create a new dataframe with just the `n` and `abool`
       columns. View the first 5 rows of this dataframe.
    1. Use `.select` to create a new dataframe with just the `group` and `abool`
       columns. View the first 5 rows of this dataframe.
    1. Use `.select` to create a new dataframe with the `group` column and the
       `abool` column renamed to `a_boolean_value`. Show the first 3 rows of
       this dataframe.
    1. Use `.select` to create a new dataframe with the `group` column and the
       `n` column renamed to `a_numeric_value`. Show the first 6 rows of this
       dataframe.


In [7]:
# Convert the pandas dataframe to a spark dataframe.
df = spark.createDataFrame(pandas_dataframe)

In [8]:
# Show the first 3 rows of the dataframe.
df.show(3)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
+--------------------+-----+-----+
only showing top 3 rows



In [9]:
# Show the first 7 rows of the dataframe.
df.show(7)

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
|  0.5323378882945463|    y|false|
|  1.3501878997225267|    z|false|
+--------------------+-----+-----+
only showing top 7 rows



In [ ]:
# What is the difference between `.show` and `.head`?

# other than the appearance of the characters, the effect is the same. 

In [10]:
# View a summary of the data using `.describe`.
df.describe().show()

+-------+------------------+-----+
|summary|                 n|group|
+-------+------------------+-----+
|  count|                20|   20|
|   mean|0.3664026449885217| null|
| stddev|0.8905322898155363| null|
|    min|-1.261605945319069|    x|
|    max|2.1503829673811126|    z|
+-------+------------------+-----+



In [11]:
# Use `.select` to create a new dataframe with just the `n` and `abool` columns.
# View the first 5 rows of this dataframe.
df1 = df.select('n', 'abool')
df1.show(5)

+--------------------+-----+
|                   n|abool|
+--------------------+-----+
|  -0.712390662050588|false|
|   0.753766378659703|false|
|-0.04450307833805...|false|
| 0.45181233874578974|false|
|  1.3451017084510097|false|
+--------------------+-----+
only showing top 5 rows



In [12]:
# Use `.select` to create a new dataframe with just the `group` and `abool` columns.
# View the first 5 rows of this dataframe.
df2 = df.select('group', 'abool')
df2.show(5)

+-----+-----+
|group|abool|
+-----+-----+
|    z|false|
|    x|false|
|    z|false|
|    y|false|
|    z|false|
+-----+-----+
only showing top 5 rows



In [13]:
# Use `.select` to create a new dataframe w/ `group` & `abool` column renamed to `a_boolean_value`.
# Show the first 3 rows of this dataframe.
col = df.abool
df3 = df.select('group', col.alias('a_boolean_value'))
df3.show(5)

+-----+---------------+
|group|a_boolean_value|
+-----+---------------+
|    z|          false|
|    x|          false|
|    z|          false|
|    y|          false|
|    z|          false|
+-----+---------------+
only showing top 5 rows



In [14]:
# Use `.select` to create a new df w/ `group` column & `n` column renamed to `a_numeric_value`.
# Show the first 6 rows of this dataframe.
col = df.n
df4 = df.select('group', col.alias('a_numeric_value'))
df4.show(6)

+-----+--------------------+
|group|     a_numeric_value|
+-----+--------------------+
|    z|  -0.712390662050588|
|    x|   0.753766378659703|
|    z|-0.04450307833805...|
|    y| 0.45181233874578974|
|    z|  1.3451017084510097|
|    y|  0.5323378882945463|
+-----+--------------------+
only showing top 6 rows



***
2. Column Manipulation

    1. Use the starter code above to re-create a spark dataframe. Store the
       spark dataframe in a varaible named `df`

    1. Use `.select` to add 4 to the `n` column. Show the results.

    1. Subtract 5 from the `n` column and view the results.

    1. Multiply the `n` column by 2. View the results along with the original
       numbers.

    1. Add a new column named `n2` that is the `n` value multiplied by -1. Show
       the first 4 rows of your dataframe. You should see the original `n` value
       as well as `n2`.

    1. Add a new column named `n3` that is the n value squared. Show the first 5
       rows of your dataframe. You should see both `n`, `n2`, and `n3`.

    1. What happens when you run the code below?

        ```python
        df.group + df.abool
        ```

    1. What happens when you run the code below? What is the difference between
       this and the previous code sample?

        ```python
        df.select(df.group + df.abool)
        ```

    1. Try adding various other columns together. What are the results of
       combining the different data types?

In [15]:
# Use `.select` to add 4 to the `n` column. Show the results.
df.select(df.n + 4).show(5)

+------------------+
|           (n + 4)|
+------------------+
|3.2876093379494122|
| 4.753766378659703|
|3.9554969216619464|
|  4.45181233874579|
|5.3451017084510095|
+------------------+
only showing top 5 rows



In [16]:
# Subtract 5 from the `n` column and view the results.
df.select(df.n - 5).show(5)

+-------------------+
|            (n - 5)|
+-------------------+
| -5.712390662050588|
| -4.246233621340297|
| -5.044503078338053|
|  -4.54818766125421|
|-3.6548982915489905|
+-------------------+
only showing top 5 rows



In [19]:
# Multiply the `n` column by 2. View the results along with the original numbers.
col = df.n * 2
df.select('n', col.alias('n x 2')).show(2)

+------------------+------------------+
|                 n|             n x 2|
+------------------+------------------+
|-0.712390662050588|-1.424781324101176|
| 0.753766378659703| 1.507532757319406|
+------------------+------------------+
only showing top 2 rows



In [20]:
# Add a new column named `n2` that is the `n` value multiplied by -1. 
# Show the first 4 rows of your dataframe. You should see the original `n` value as well as `n2`.
col = df.n * (-1)
df2 = df.select('n', col.alias('n2'))
df2.show(4)

+--------------------+--------------------+
|                   n|                  n2|
+--------------------+--------------------+
|  -0.712390662050588|   0.712390662050588|
|   0.753766378659703|  -0.753766378659703|
|-0.04450307833805...|0.044503078338053455|
| 0.45181233874578974|-0.45181233874578974|
+--------------------+--------------------+
only showing top 4 rows



In [22]:
# Add a new column named `n3` that is the n value squared. Show the first 5 rows of your dataframe.
# You should see `n`, `n2`, and `n3`.
col = df.n ** 2
df3 = df2.select('n', 'n2', col.alias('n3'))
df3.show(5)

+--------------------+--------------------+--------------------+
|                   n|                  n2|                  n3|
+--------------------+--------------------+--------------------+
|  -0.712390662050588|   0.712390662050588|   0.507500455376875|
|   0.753766378659703|  -0.753766378659703|  0.5681637535977627|
|-0.04450307833805...|0.044503078338053455|0.001980523981562...|
| 0.45181233874578974|-0.45181233874578974| 0.20413438944294027|
|  1.3451017084510097| -1.3451017084510097|  1.8092986060778251|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [23]:
# What happens when you run the code below?
df.group + df.abool
# group and abool columns are fused into a single column

Column<'(group + abool)'>

In [24]:
# What happens when you run the code below? 
# What is the difference between this and the previous code sample?
df.select(df.group + df.abool)
# it throws an error due to incompatabilities between differing dtypes

AnalysisException: cannot resolve '(CAST(group AS DOUBLE) + abool)' due to data type mismatch: differing types in '(CAST(group AS DOUBLE) + abool)' (double and boolean).;
'Project [unresolvedalias((cast(group#20 as double) + abool#21), Some(org.apache.spark.sql.Column$$Lambda$3355/0x00000008013e9840@2d7475b5))]
+- LogicalRDD [n#19, group#20, abool#21], false


In [25]:
# Try adding various other columns together. 
# What are the results of combining the different data types?
df3.select(df3.n2 + df3.n3).show(5)
# when the datatypes match, all is well. 

+--------------------+
|           (n2 + n3)|
+--------------------+
|   1.219891117427463|
|-0.18560262506194025|
|0.046483602319616374|
|-0.24767794930284948|
|  0.4641968976268154|
+--------------------+
only showing top 5 rows



***
3. Type casting

    1. Use the starter code above to re-create a spark dataframe.

    1. Use `.printSchema` to view the datatypes in your dataframe.

    1. Use `.dtypes` to view the datatypes in your dataframe.

    1. What is the difference between the two code samples below?

        ```python
        df.abool.cast('int')
        ```

        ```python
        df.select(df.abool.cast('int')).show()
        ```

    1. Use `.select` and `.cast` to convert the `abool` column to an integer
       type. View the results.
    1. Convert the `group` column to a integer data type and view the results.
       What happens?
    1. Convert the `n` column to a integer data type and view the results. What
       happens?
    1. Convert the `abool` column to a string data type and view the results.
       What happens?

In [26]:
# in case I messed any original content up
df = spark.createDataFrame(pandas_dataframe)

In [28]:
# Use `.printSchema` to view the datatypes in your dataframe.
df.printSchema()

root
 |-- n: double (nullable = true)
 |-- group: string (nullable = true)
 |-- abool: boolean (nullable = true)



In [31]:
# Use `.dtypes` to view the datatypes in your dataframe.
df.dtypes

[('n', 'double'), ('group', 'string'), ('abool', 'boolean')]

In [32]:
# What is the difference between the two code samples below?
df.abool.cast('int')

Column<'CAST(abool AS INT)'>

In [33]:
df.select(df.abool.cast('int')).show() # this does the same, casting as an int, but gives rundown

+-----+
|abool|
+-----+
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    1|
|    1|
|    0|
|    0|
|    1|
|    1|
|    0|
|    0|
|    0|
|    1|
|    0|
|    1|
+-----+



In [ ]:
# Use `.select` and `.cast` to convert the `abool` column to an integer type. View the results.

# that was accomplished in the previous code block

In [35]:
# Convert the `group` column to a integer data type and view the results. What happens?
# they all became null because strings cannot be converted to integers
df.select(df.group.cast('int')).show(5)

+-----+
|group|
+-----+
| null|
| null|
| null|
| null|
| null|
+-----+
only showing top 5 rows



In [36]:
# Convert the `n` column to a integer data type and view the results. What happens?
# rounds to nearest integer
df.select(df.n.cast('int')).show()

+---+
|  n|
+---+
|  0|
|  0|
|  0|
|  0|
|  1|
|  0|
|  1|
|  0|
|  1|
| -1|
|  0|
| -1|
|  0|
|  0|
|  0|
|  0|
|  0|
|  2|
|  0|
|  0|
+---+



In [38]:
# Convert the `abool` column to a string data type and view the results. What happens?
# String representation of the boolean value
df.select(df.abool.cast('string')).show(10)

+-----+
|abool|
+-----+
|false|
|false|
|false|
|false|
|false|
|false|
|false|
|false|
| true|
| true|
+-----+
only showing top 10 rows



***
4. Built-in Functions

    1. Use the starter code above to re-create a spark dataframe.
    1. Import the necessary functions from `pyspark.sql.functions`
    1. Find the highest `n` value.
    1. Find the lowest `n` value.
    1. Find the average `n` value.
    1. Use `concat` to change the `group` column to say, e.g. "Group: x" or
       "Group: y"
    1. Use `concat` to combine the `n` and `group` columns to produce results
       that look like this: "x: -1.432" or "z: 2.352"

In [39]:
# Find the highest `n` value.
df.select(max(df.n)).show()

+------------------+
|            max(n)|
+------------------+
|2.1503829673811126|
+------------------+



In [40]:
# Find the lowest `n` value.
df.select(min(df.n)).show()

+------------------+
|            min(n)|
+------------------+
|-1.261605945319069|
+------------------+



In [41]:
# Find the average `n` value.
df.select(mean(df.n)).show()

+------------------+
|            avg(n)|
+------------------+
|0.3664026449885217|
+------------------+



In [42]:
# Use `concat` to change the `group` column to say, e.g. "Group: x" or "Group: y"
df.select(concat(lit('Group:'), df.group).alias('new_group')).show(7)

+---------+
|new_group|
+---------+
|  Group:z|
|  Group:x|
|  Group:z|
|  Group:y|
|  Group:z|
|  Group:y|
|  Group:z|
+---------+
only showing top 7 rows



In [43]:
# Use `concat` to combine the `n` and `group` columns to produce results that look like this:
# "x: -1.432" or "z: 2.352"
df.select(concat(df.group, lit(':'), df.n).alias('new_group')).show(5)

+--------------------+
|           new_group|
+--------------------+
|z:-0.712390662050588|
| x:0.753766378659703|
|z:-0.044503078338...|
|y:0.4518123387457...|
|z:1.3451017084510097|
+--------------------+
only showing top 5 rows



***
5. When / Otherwise

    1. Use the starter code above to re-create a spark dataframe.
    1. Use `when` and `.otherwise` to create a column that contains the text "It
       is true" when `abool` is true and "It is false"" when `abool` is false.
    1. Create a column that contains 0 if n is less than 0, otherwise, the
       original n value.

In [44]:
# Use `when` and `.otherwise` to create a column that contains the text 
# "It is true" when `abool` is true and "It is false"" when `abool` is false.
df.select(
    'abool',
    (when(df.abool > True, 'It is true')
     .otherwise('It is false')
     .alias('abool_desc'))
).show(5)

+-----+-----------+
|abool| abool_desc|
+-----+-----------+
|false|It is false|
|false|It is false|
|false|It is false|
|false|It is false|
|false|It is false|
+-----+-----------+
only showing top 5 rows



In [45]:
# Create a column that contains 0 if n is less than 0, otherwise, the original n value.
df.select(
    'n',
    (when(df.n < 0, 0)
     .otherwise(df.n)
     .alias('new_n'))
).show(5)

+--------------------+-------------------+
|                   n|              new_n|
+--------------------+-------------------+
|  -0.712390662050588|                0.0|
|   0.753766378659703|  0.753766378659703|
|-0.04450307833805...|                0.0|
| 0.45181233874578974|0.45181233874578974|
|  1.3451017084510097| 1.3451017084510097|
+--------------------+-------------------+
only showing top 5 rows



***
6. Filter / Where

    1. Use the starter code above to re-create a spark dataframe.
    1. Use `.filter` or `.where` to select just the rows where the group is `y`
       and view the results.
    1. Select just the columns where the `abool` column is false and view the
       results.
    1. Find the columns where the `group` column is *not* `y`.
    1. Find the columns where `n` is positive.
    1. Find the columns where `abool` is true and the `group` column is `z`.
    1. Find the columns where `abool` is true or the `group` column is `z`.
    1. Find the columns where `abool` is false and `n` is less than 1
    1. Find the columns where `abool` is false or `n` is less than 1

In [47]:
# Use `.filter` or `.where` to select just the rows where the group is `y` and view the results.
df.where(df.group == 'y').show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
| 0.45181233874578974|    y|false|
|  0.5323378882945463|    y|false|
| -1.0453771305385342|    y| true|
|  -1.261605945319069|    y|false|
|  0.5628467852810314|    y| true|
|-0.24332625188556253|    y| true|
|  0.9137407048596775|    y|false|
|  2.1503829673811126|    y| true|
+--------------------+-----+-----+



In [48]:
# Select just the columns where the `abool` column is false and view the results.
df.where(df.abool == 'false').show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
|  0.5323378882945463|    y|false|
|  1.3501878997225267|    z|false|
|  0.8612113741693206|    x|false|
| -0.7889890249515489|    x|false|
|  -1.261605945319069|    y|false|
|  0.9137407048596775|    y|false|
| 0.31735092273633597|    x|false|
| 0.12730328020698067|    z|false|
|  0.6062886568962988|    x|false|
+--------------------+-----+-----+



In [54]:
# Find the columns where the `group` column is *not* `y`.
df.where((df.group == 'x') | (df.group == 'z')).show()
# this is a cheating approach, but ~ wasn't working for me. 
# might return to this later with a question. 

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
|  1.3451017084510097|    z|false|
|  1.3501878997225267|    z|false|
|  0.8612113741693206|    x|false|
|  1.4786857374358966|    z| true|
| -0.7889890249515489|    x|false|
| 0.31735092273633597|    x|false|
| 0.12730328020698067|    z|false|
|  0.6062886568962988|    x|false|
|-0.02677164998644...|    x| true|
+--------------------+-----+-----+



In [55]:
# Find the columns where `n` is positive.
df.where(df.n > 0).show()

+-------------------+-----+-----+
|                  n|group|abool|
+-------------------+-----+-----+
|  0.753766378659703|    x|false|
|0.45181233874578974|    y|false|
| 1.3451017084510097|    z|false|
| 0.5323378882945463|    y|false|
| 1.3501878997225267|    z|false|
| 0.8612113741693206|    x|false|
| 1.4786857374358966|    z| true|
| 0.5628467852810314|    y| true|
| 0.9137407048596775|    y|false|
|0.31735092273633597|    x|false|
|0.12730328020698067|    z|false|
| 2.1503829673811126|    y| true|
| 0.6062886568962988|    x|false|
+-------------------+-----+-----+



In [56]:
# Find the columns where `abool` is true AND the `group` column is `z`.
df.where((df.abool == 'true') & (df.group == 'z')).show()

+------------------+-----+-----+
|                 n|group|abool|
+------------------+-----+-----+
|1.4786857374358966|    z| true|
+------------------+-----+-----+



In [57]:
# Find the columns where `abool` is true OR the `group` column is `z`.
df.where((df.abool == 'true') | (df.group == 'z')).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|-0.04450307833805...|    z|false|
|  1.3451017084510097|    z|false|
|  1.3501878997225267|    z|false|
|  1.4786857374358966|    z| true|
| -1.0453771305385342|    y| true|
|  0.5628467852810314|    y| true|
|-0.24332625188556253|    y| true|
| 0.12730328020698067|    z|false|
|  2.1503829673811126|    y| true|
|-0.02677164998644...|    x| true|
+--------------------+-----+-----+



In [58]:
# Find the columns where `abool` is false AND `n` is less than 1
df.where((df.abool == 'false') & (df.n < 1)).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  0.5323378882945463|    y|false|
|  0.8612113741693206|    x|false|
| -0.7889890249515489|    x|false|
|  -1.261605945319069|    y|false|
|  0.9137407048596775|    y|false|
| 0.31735092273633597|    x|false|
| 0.12730328020698067|    z|false|
|  0.6062886568962988|    x|false|
+--------------------+-----+-----+



In [59]:
# Find the columns where `abool` is false OR `n` is less than 1
df.where((df.abool == 'false') | (df.n < 1)).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  -0.712390662050588|    z|false|
|   0.753766378659703|    x|false|
|-0.04450307833805...|    z|false|
| 0.45181233874578974|    y|false|
|  1.3451017084510097|    z|false|
|  0.5323378882945463|    y|false|
|  1.3501878997225267|    z|false|
|  0.8612113741693206|    x|false|
| -1.0453771305385342|    y| true|
| -0.7889890249515489|    x|false|
|  -1.261605945319069|    y|false|
|  0.5628467852810314|    y| true|
|-0.24332625188556253|    y| true|
|  0.9137407048596775|    y|false|
| 0.31735092273633597|    x|false|
| 0.12730328020698067|    z|false|
|  0.6062886568962988|    x|false|
|-0.02677164998644...|    x| true|
+--------------------+-----+-----+



***
7. Sorting

    1. Use the starter code above to re-create a spark dataframe.
    1. Sort by the `n` value.
    1. Sort by the `group` value, both ascending and descending.
    1. Sort by the group value first, then, within each group, sort by `n`
       value.
    1. Sort by `abool`, `group`, and `n`. Does it matter in what order you
       specify the columns when sorting?

In [61]:
# Sort by the `n` value.
df.orderBy(df.n.desc()).show(5)

+------------------+-----+-----+
|                 n|group|abool|
+------------------+-----+-----+
|2.1503829673811126|    y| true|
|1.4786857374358966|    z| true|
|1.3501878997225267|    z|false|
|1.3451017084510097|    z|false|
|0.9137407048596775|    y|false|
+------------------+-----+-----+
only showing top 5 rows



In [65]:
# Sort by the `group` value, both ascending and descending.
print(f'ascending: {df.orderBy(df.group.asc()).show(10)}')
print(f'descending: {df.orderBy(df.group.desc()).show(10)}')

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|-0.02677164998644...|    x| true|
| -0.7889890249515489|    x|false|
|  0.8612113741693206|    x|false|
|  0.6062886568962988|    x|false|
| 0.31735092273633597|    x|false|
|   0.753766378659703|    x|false|
|-0.24332625188556253|    y| true|
|  -1.261605945319069|    y|false|
| -1.0453771305385342|    y| true|
|  2.1503829673811126|    y| true|
+--------------------+-----+-----+
only showing top 10 rows

ascending: None
+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
|  1.4786857374358966|    z| true|
| 0.12730328020698067|    z|false|
|-0.04450307833805...|    z|false|
|  -0.712390662050588|    z|false|
|  1.3501878997225267|    z|false|
|  1.3451017084510097|    z|false|
|  0.5323378882945463|    y|false|
|  -1.261605945319069|    y|false|
| 0.45181233874578974|    y|false|
| -1.0453771305385342|    y| true|
+------------

In [69]:
# Sort by the group value first, then, within each group, sort by `n` value.
df.orderBy(df.group.asc(), df.n.asc()).show()

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
| -0.7889890249515489|    x|false|
|-0.02677164998644...|    x| true|
| 0.31735092273633597|    x|false|
|  0.6062886568962988|    x|false|
|   0.753766378659703|    x|false|
|  0.8612113741693206|    x|false|
|  -1.261605945319069|    y|false|
| -1.0453771305385342|    y| true|
|-0.24332625188556253|    y| true|
| 0.45181233874578974|    y|false|
|  0.5323378882945463|    y|false|
|  0.5628467852810314|    y| true|
|  0.9137407048596775|    y|false|
|  2.1503829673811126|    y| true|
|  -0.712390662050588|    z|false|
|-0.04450307833805...|    z|false|
| 0.12730328020698067|    z|false|
|  1.3451017084510097|    z|false|
|  1.3501878997225267|    z|false|
|  1.4786857374358966|    z| true|
+--------------------+-----+-----+



In [70]:
# Sort by `abool`, `group`, and `n`. 
# Does it matter in what order you specify the columns when sorting?
df.orderBy(df.abool.asc(),df.group.asc(), df.n.asc()).show()
# just comparing this to the former does show that order matters. 
# it follows a prioritization
# the first one is ordered, then the following without changing the initial ordering and so on 

+--------------------+-----+-----+
|                   n|group|abool|
+--------------------+-----+-----+
| -0.7889890249515489|    x|false|
| 0.31735092273633597|    x|false|
|  0.6062886568962988|    x|false|
|   0.753766378659703|    x|false|
|  0.8612113741693206|    x|false|
|  -1.261605945319069|    y|false|
| 0.45181233874578974|    y|false|
|  0.5323378882945463|    y|false|
|  0.9137407048596775|    y|false|
|  -0.712390662050588|    z|false|
|-0.04450307833805...|    z|false|
| 0.12730328020698067|    z|false|
|  1.3451017084510097|    z|false|
|  1.3501878997225267|    z|false|
|-0.02677164998644...|    x| true|
| -1.0453771305385342|    y| true|
|-0.24332625188556253|    y| true|
|  0.5628467852810314|    y| true|
|  2.1503829673811126|    y| true|
|  1.4786857374358966|    z| true|
+--------------------+-----+-----+



***
8. Aggregating

    1. What is the average `n` value for each group in the `group` column?
    1. What is the maximum `n` value for each group in the `group` column?
    1. What is the minimum `n` value by `abool`?
    1. What is the average `n` value for each unique combination of the `group`
       and `abool` column?

In [72]:
# What is the average `n` value for each group in the `group` column?
df.groupBy('group').agg(mean('n')).show()

+-----+------------------+
|group|            avg(n)|
+-----+------------------+
|    x|0.2871427762539448|
|    z| 0.590730814237962|
|    y| 0.257601419602374|
+-----+------------------+



In [73]:
# What is the maximum `n` value for each group in the `group` column?
df.groupBy('group').agg(max('n')).show()

+-----+------------------+
|group|            max(n)|
+-----+------------------+
|    x|0.8612113741693206|
|    z|1.4786857374358966|
|    y|2.1503829673811126|
+-----+------------------+



In [74]:
# What is the minimum `n` value by `abool`?
df.groupBy('abool').agg(min('n')).show()

+-----+-------------------+
|abool|             min(n)|
+-----+-------------------+
|false| -1.261605945319069|
| true|-1.0453771305385342|
+-----+-------------------+



In [75]:
# What is the average `n` value for each unique combination of the `group and `abool` column?
df.groupBy('group', 'abool').agg(mean('n')).show()

+-----+-----+--------------------+
|group|abool|              avg(n)|
+-----+-----+--------------------+
|    z|false| 0.41313982959837514|
|    x|false|  0.3499256615020219|
|    y|false| 0.15907124664523611|
|    y| true| 0.35613159255951177|
|    z| true|  1.4786857374358966|
|    x| true|-0.02677164998644...|
+-----+-----+--------------------+



22/05/18 16:02:11 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 959402 ms exceeds timeout 120000 ms
22/05/18 16:02:11 WARN SparkContext: Killing executors is not supported by current scheduler.


***
9. Spark SQL

    1. Use the starter code above to re-create a spark dataframe.
    1. Turn your dataframe into a table that can be queried with spark SQL. Name
       the table `my_df`. Answer the rest of the questions in this section with
       a spark sql query (`spark.sql`) against `my_df`. After each step, view
       the first 7 records from the dataframe.
    1. What happens if you make a SQL syntax error in your query?
    1. Write a query that shows all of the columns from your dataframe.
    1. Write a query that shows just the `n` and `abool` columns from the
       dataframe.
    1. Write a query that shows just the `n` and `group` columns. Rename the
       `group` column to `g`.
    1. Write a query that selects `n`, and creates two new columns: `n2`, the
       original `n` values halved, and `n3`: the original n values minus 1.

In [ ]:
# Turn your dataframe into a table that can be queried with spark SQL. Name the table `my_df`. 
# Answer the rest of the questions with a spark sql query (`spark.sql`) against `my_df`.
# After each step, view the first 7 records from the dataframe.

In [ ]:
# What happens if you make a SQL syntax error in your query?

In [ ]:
# Write a query that shows all of the columns from your dataframe.

In [ ]:
# Write a query that shows just the `n` and `abool` columns from the dataframe.

In [ ]:
# Write a query that shows just the `n` and `group` columns. Rename the `group` column to `g`.

In [ ]:
# Write a query that selects `n`, and creates two new columns: 
# `n2`, the original `n` values halved, and `n3`: the original n values minus 1.